Zonas de subducción:
* [Global correlations between maximum magnitudes of subduction zone interface thrust earthquakes and physical parameters of subduction zonesm](https://www.sciencedirect.com/science/article/pii/S0031920113001465#b0695)

In [1]:
# stdlib imports
from datetime import datetime
from time import time
import pickle

# Third party imports
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth
import pandas as pd

# Local imports
from libcomcat.dataframes import get_history_data_frame
from libcomcat.search import (get_event_by_id, get_authoritative_info, search)

# Data acquisition and formatting

In [2]:
def date_extracter(date):
    formatted = date.strftime("%d-%m-%Y (%H:%M:%S.%f)")
    string_subsect = formatted.split(" ")
    # Date formatting
    date = string_subsect[0]
    date = date.split("-")
    #print(date)
    # Hour formatting
    hour = string_subsect[1]
    hour = hour[1:-2] # To cut the parenthesis
    hour = hour.split(":")
    hour[2] = hour[2][0:2]
    # Proper parsing
    parsed = list()
    for item in date:
        parsed.append(item)
    for item in hour:
        parsed.append(item)
    #print(parsed)
    return parsed
    #return date

def event_generator(date, lat, lon, depth, magnitude):
    #print(date)
    day = date[0]
    month = date[1]
    year = date[2]
    hour = date[3]
    minute = date[4]
    second = date[5]
    event = [year, month, day, hour, minute, second, lat, lon, depth, magnitude]
    return event

def event_extracter(events):
    event_formatted = list()
    for event in events:
        #print(event)
        for key, value in event.items():
            #print(key, value)
            if key == "time":
                #print("ASDF")
                #print(type(value))
                date = date_extracter(value)
                #print("DATE")
                #print(date)
            if key == "latitude":
                lat = value
                #print("LAT")
            if key == "longitude":
                lon = value
                #print("LON")
            if key == "depth":
                depth = value
                #print("DEPTH")
            if key == "magnitude":
                magnitude = value
                #print("MAG")
            #print(date)
            #print(lat)
            #print(lon)
            #print(depth)
            #print(magnitude)
        ev = event_generator(date, lat, lon, depth, magnitude)
        event_formatted.append(ev)
    return event_formatted

def do_query(bounds, minmag, maxmag):
    data = []
    for i in range(0,18):
        stime = datetime(2000+i,1,1)
        etime = datetime(2001+i,1,1)
        eventlist = search(starttime=stime,
                      endtime=etime,
                      minlatitude=bounds[2],
                      maxlatitude=bounds[3],
                      minlongitude=bounds[0],
                      maxlongitude=bounds[1],
                      minmagnitude=minmag,
                      maxmagnitude=maxmag)
        data = data + eventlist
    return data

def event_parser(event):
    year = int(event[0])
    month = int(event[1])
    day = int(event[2])
    hour = int(event[3])
    minute = int(event[4])
    second = int(event[5])
    lat = float(event[6])
    lon = float(event[7])
    depth = float(event[8])
    magnitude = float(event[9])
    date = datetime(year, month, day, hour, minute, second).strftime("%d-%m-%Y %H:%M:%S")
    #print(date)
    ev = [date, lat, lon, depth, magnitude]
    return ev
    
def second_formatting(events):
    formatted = list()
    count = 0
    for event in events:
        new_ev = list()
        #if (count == 10):
            #break
        #else:
        new_ev = event_parser(event)
        formatted.append(new_ev)
        #print(new_ev)
        #count +=1
    return formatted

# Chile
Query to the API

In [3]:
# VERY approximate Chile bounds
bounds = [-86.748, -66.885, -54.858, -16.774]

# magnitude range
minmag = 3.0
maxmag = 9.9

data = do_query(bounds, minmag, maxmag)
manipulate = [event.toDict() for event in data]
event_formatted = event_extracter(manipulate)
refreshed_events = second_formatting(event_formatted)
refreshed_events.sort(key = lambda date: datetime.strptime(date[0], '%d-%m-%Y %H:%M:%S'))
pickled_data = open("chile_data.pkl", "wb")
pickle.dump(refreshed_events, pickled_data)

# Japan

* Subregion 1:
    - [28.466, 34.597] Latitude
    - [128.167, 147.612] Longitude
    
* Subregion 2:
    - [34.597, 41.509] Latitude
    - [130.781, 149.326] Longitude
    
* Subregion 3:
    - [41.509, 45.675] Latitude
    - [138.955, 151.084] Longitude
    
Se toma de referencia el área en base a diferentes publicaciones.
* https://www.u-tokyo.ac.jp/en/whyutokyo/wj_001.html
* [Morphology of the subducting slab boundary in the northeastern Japan arc](https://sci-hub.tw/10.1016/s0031-9201(96)03258-x)
* [National  Seismic  Hazard  Maps  of  Japan](http://www.eri.u-tokyo.ac.jp/BERI/pdf/IHO81304.pdf)

In [5]:
# VERY approximate Japan bounds
# bounds = minlong, maxlong, minlat, maxlat
bounds_1 = [128.167, 151.084, 28.466, 34.597]
bounds_2 = [130.781, 151.084, 34.597, 41.509]
bounds_3 = [138.955, 151.084, 41.509, 45.675]

# magnitude range
minmag = 3.0
maxmag = 9.9

data_1 = do_query(bounds_1, minmag, maxmag)
data_2 = do_query(bounds_2, minmag, maxmag)
data_3 = do_query(bounds_3, minmag, maxmag)
data = data_1 + data_2 + data_3
    
manipulate = [event.toDict() for event in data]
event_formatted = event_extracter(manipulate)
refreshed_events = second_formatting(event_formatted)
refreshed_events.sort(key = lambda date: datetime.strptime(date[0], '%d-%m-%Y %H:%M:%S'))
pickled_data = open("japan_data.pkl", "wb")
pickle.dump(refreshed_events, pickled_data)

# Sumatra

Se toma de referencia el área en base a diferentes publicaciones.
* http://www.tectonics.caltech.edu/outreach/highlights/sumatra/why.html

# Alaska